In [ ]:
# import
from yolov8.nn.autobackend import AutoBackend
from yolov8.utils.ops import non_max_suppression
import torchvision
import torchvision.datasets
import torchvision.transforms as transforms
import torch
from tqdm import tqdm
from torch.utils.data import Dataset
import cv2
import numpy as np

# Global variables
COCO_CLASS_NAMES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
            'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
            'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
            'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
            'tennis racket', 'bzottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
            'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
            'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
            'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
            'hair drier', 'toothbrush']
COCO_CLASS_INDEX = {label: i for i, label in enumerate(COCO_CLASS_NAMES)}
COCO_CLASS_NAMES_SORTED = sorted(COCO_CLASS_NAMES)
PRED_LABEL = {label: [] for label in COCO_CLASS_NAMES}
COCO_ROOT = "C:/Users/phiho/Projects/Dataset/MS-COCO/COCO_81/coco81/"

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Check device: ', device)

# Model
ROOT = './yolov8'  # YOLOv8 root directory
imgsz=(608, 608)
resize = torchvision.transforms.Resize(imgsz)
weights = ROOT + '/' + "yolov8s.pt"
data = ROOT + '/' + "data/coco128.yaml"
imgsz=(320, 320)
resize = torchvision.transforms.Resize(imgsz)
model = AutoBackend(weights, device=device)
stride, names, pt = model.stride, model.names, model.pt
conf_thres = 0
print('Load model:', weights)

# Dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None, device='cpu'):
        self.dataset = torchvision.datasets.ImageFolder(root_dir, transform=transform)
        self.device = device
        self.img_paths = [path for path, _ in self.dataset.samples]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img, label = self.dataset[idx]
        img = cv2.cvtColor(img.cpu().numpy().transpose((1, 2, 0)), cv2.COLOR_RGB2BGR)
        img = torch.from_numpy(img.transpose(2, 0, 1)).float().unsqueeze(0)
        img = img.to(self.device)
        
        return img, label, img_path
    
coco_dataset = CustomDataset(root_dir=COCO_ROOT, 
                             transform=transforms.ToTensor(), 
                             device=device)
print('Load dataset:', coco_dataset.__len__(), 'images')

# Utils
def get_predict(im, conf_thres=conf_thres):
    batch = 1
    model.warmup(imgsz=(1 if pt or model.triton else batch, 3, *imgsz))
    im = resize(im)
    im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
    pred = model(im)
    pred = non_max_suppression(pred, conf_thres=conf_thres)
    res = np.zeros(80)
    for i, det in enumerate(pred):
        if len(det):
            for *xyxy, conf, cls in reversed(det):
                c = int(cls)
                confidence = float(conf)
                if res[c] < confidence:
                    res[c] = confidence
    return np.argmax(res)

In [ ]:
import json

def load_json_from_txt(path):
    with open(path, 'r') as file:
        data = file.read()
    return json.loads(data)

PRED_LABEL = load_json_from_txt('C:/Users/phiho/Projects/GAN_Yolov8/analysis/PRED_LABEL.txt')

In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import os

# Get confusion matrix
for i, data in tqdm(enumerate(coco_dataset, start=0)):
    image, label_gt, im_path = data   
    true_reidx = COCO_CLASS_INDEX[COCO_CLASS_NAMES_SORTED[label_gt]]
    label_pred = get_predict(image)
    PRED_LABEL[COCO_CLASS_NAMES[label_pred]].append(im_path)

# Save prediction to txt
with open('C:/Users/phiho/Projects/GAN_Yolov8/analysis/PRED_LABEL.txt', 'w') as file:
    json.dump(PRED_LABEL, file)

# Save confusion matrix to excel
cm = np.zeros((80, 80), dtype=float)

for lb in COCO_CLASS_NAMES:
    label_pred = COCO_CLASS_INDEX[lb]
    for im_path in PRED_LABEL[lb]:
        label_true = COCO_CLASS_INDEX[os.path.basename(os.path.dirname(im_path))]
        cm[label_true][label_pred] += 1.0

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm_with_label = []

# Row 1
row_1 = [""]
for lb in COCO_CLASS_NAMES:
    row_1.append(lb)
cm_with_label.append(row_1)

# Row i
for i, lb in enumerate(COCO_CLASS_NAMES):
    row_i = [lb]
    for dt in cm_normalized[i]:
        row_i.append(dt)
    cm_with_label.append(row_i)

df = pd.DataFrame(cm_with_label)
df.to_excel('C:/Users/phiho/Projects/GAN_Yolov8/analysis/ConfusionMatrix.xlsx', sheet_name='All', index=False)

In [ ]:
PRED_LABEL

In [ ]:
import os

COUNT_ALL_LABEL = {label: 0 for label in COCO_CLASS_NAMES}
c_ = 0
for f1 in os.listdir(COCO_ROOT):
    f1 = os.path.join(COCO_ROOT, f1)
    for f2 in os.listdir(f1):
        COUNT_ALL_LABEL[f2] += len(os.listdir(os.path.join(f1, f2)))

print('person : ', COUNT_ALL_LABEL['person'])
print('dog : ', COUNT_ALL_LABEL['cat'])
print('cat : ', COUNT_ALL_LABEL['cat'])
print('bear : ', COUNT_ALL_LABEL['bear'])

In [ ]:
import shutil
import os
from sklearn.model_selection import train_test_split

PRED_LABEL = load_json_from_txt('C:/Users/phiho/Projects/GAN_Yolov8/analysis/PRED_LABEL.txt')
LABELS_PICK = ['person', 'dog', 'cat', 'bear']
DATA_100 = {lb : [] for lb in LABELS_PICK}

for lb in LABELS_PICK:
    for im_path in PRED_LABEL[lb]:
        if os.path.basename(os.path.dirname(im_path)) == lb:
            DATA_100[lb].append(im_path)
DATA_100['person'] = DATA_100['person'][0:2000]

with open('C:/Users/phiho/Projects/GAN_Yolov8/analysis/DATA_100.txt', 'w') as file:
    json.dump(DATA_100, file)

new_path = 'C:/Users/phiho/Projects/GAN_Yolov8/Dataset/dog_cat_person_bear'
os.makedirs(new_path)

f_name = ['train', 'test']
for lb in LABELS_PICK:
    image_paths = DATA_100[lb]
    paths = train_test_split(image_paths, test_size=0.2, random_state=42)

    print(f"Label: {lb}")
    print(f"       {f_name[0]}: {len(paths[0])} images")
    print(f"       {f_name[1]}: {len(paths[1])} images")

    for i, fs in enumerate(paths):
        for im_path in fs:
            im_name = os.path.basename(im_path)
            src = im_path
            des = os.path.join(new_path, f_name[i], lb, im_name)
            os.makedirs(os.path.join(new_path, f_name[i], lb), exist_ok=True)
            shutil.copy(src, des)